In [5]:
import numpy as np
from matplotlib import pyplot as plt
import os
from pathlib import Path
import cv2

In [6]:
p = Path("/../../users/ankur/desktop/ml/flowers")
dirs = p.glob("*")

labels_dict = {"daisy":0,
              "dandelion":1,
              "rose":2,
              "sunflower":3,
              "tulip":4}
image_data=[]
labels=[]
    
for folder_dir in dirs:
#     print(folder)
    label = str(folder_dir).split("\\")[-1]
#     print(label)
 
    for img_path in folder_dir.glob("*.jpg"):
#         print(type(img_path))
        img_path=str(img_path).replace('\\','/')
#         print(img_path)
        img = cv2.imread(img_path)
#         print((img))
#         print("---------------------------------")
        img = cv2.resize(img,(64,64))
#         print((img))
#         plt.imshow(img)
#         print(img.shape)
#         img_array = image.img_to_array(img)
        image_data.append(img)
        labels.append(labels_dict[label])

In [7]:
print(len(image_data))
print(len(labels))

4323
4323


In [8]:
image_data = np.array(image_data)
labels = np.array(labels)
print(image_data.shape,labels.shape)

(4323, 64, 64, 3) (4323,)


In [9]:
import random
combined = list(zip(image_data,labels))
random.shuffle(combined)

image_data[:],labels[:] = zip(*combined)
# plt.imshow(image_data[428])

In [25]:
class svm:
    def __init__(self,c=1.0):
        self.c=c
        self.w=0
        self.b=0
        
    def hingeLoss(self,w,b,x,y):
        loss = 0.0
        loss += .5*np.dot(w,w.T)
        m=x.shape[0]
        for i in range(m):
            ti = y[i]*(np.dot(w,x[i].T)+b)
            loss += self.c * max(0,1-ti)
#             print(loss)
            
        return loss[0][0]
    
    def fit(self,x,y,batch_size=100,learning_rate=0.00001,maxItr=5000):
        no_of_features = x.shape[1]
        no_of_samples = x.shape[0]
        
        n=learning_rate
        c=self.c
#         batch_size=100
        w=np.zeros((1,no_of_features))
        b=0
        
#         print(self.hingeLoss(w,b,x,y)[0][0])
        
        losses=[]
        
        for i in range(maxItr):
            
            l = self.hingeLoss(w,b,x,y)
#             print(l)
            losses.append(l)
            ids = np.arange(no_of_samples)
            np.random.shuffle(ids)
            for batch_start in range(0,no_of_samples,batch_size):
                gradw=0
                gradb=0
                
                for j in range(batch_start,batch_start+batch_size):
                    if j<no_of_samples:
                        i=ids[j]
                    
                        ti = y[i]*(np.dot(w,x[i].T)+b)
                        
                        if(ti>1):
                            gradw += 0
                            gradb += 0
                        else:
                            gradw += c*y[i]*x[i]
                            gradb += c*y[i]
                            
                w = w - n*w  + n*gradw
                b = b + n*gradb
                
        self.w=w
        self.b=b
        return w,b,losses
    

In [26]:
m = image_data.shape[0]
image_data = image_data.reshape(m,-1)
print(image_data.shape)

(4323, 12288)


In [27]:
classes = len(np.unique(labels))
print(classes)

5


In [28]:
def classWiseData(x,y):
    data = {}
    
    for i in range(classes):
        data[i]=[]
        
    for i in range(x.shape[0]):
        data[y[i]].append(x[i])
        
    for k in data.keys():
        data[k] = np.array(data[k])
        
    return data

In [29]:
data = classWiseData(image_data,labels)

In [30]:
def getDataPairForSvm(d1,d2):
    
    l1,l2 = d1.shape[0],d2.shape[0]
    
    samples = l1+l2
    features = d1.shape[1]
    
    data_pair = np.zeros((samples,features))
    data_labels = np.zeros(samples)
    
    data_pair[:l1,:] = d1
    data_pair[l1:,:] = d2
    
    data_labels[:l1] = -1
    data_labels[l1:] = +1
    
    return data_pair,data_labels

In [31]:
mySVM = svm()
def trainSVM(x,y):
    
    svm_classifiers = {}
    for i in range(classes):
        svm_classifiers[i] = {}
        for j in range(i+1,classes):
            xpair,ypair = getDataPairForSvm(data[i],data[j])
            wts,b,loss = mySVM.fit(xpair,ypair)
            svm_classifiers[i][j] = (wts,b)
#             plt.plot(loss)
#             plt.show() 
    return svm_classifiers

In [32]:
svm_classifiers = trainSVM(image_data,labels)

In [33]:
print(svm_classifiers[0][1][0].shape)

(1, 12288)


In [34]:
def binaryPredict(x,w,b):
    z = np.dot(x,w.T)+b
    if z>=0:
        return 1
    else :
        return -1

In [35]:
def predict(x):
    count = np.zeros(classes)
    
    for i in range(classes):
        for j in range(i+1,classes):
            w,b = svm_classifiers[i][j]
            z = binaryPredict(x,w,b)
            if z==1 :
                count[j] +=1
            else :
                count[i] +=1
    final_prediction = np.argmax(count)
#     print(count)
    return final_prediction

In [36]:
# print(image_data.shape)
print(predict(image_data[1000]))
print(labels[1000])

2
2


In [37]:
def accuracy(x,y):
    count =0
    for i in range(x.shape[0]):
        prediction = predict(x[i])
        if(prediction == y[i]):
            count +=1
    return count/x.shape[0]

    

In [38]:
accuracy(image_data,labels)

0.6081424936386769

### SKlearn

In [24]:
from sklearn import svm
svm_classifier = svm.SVC(kernel='linear',C=1.0)
svm_classifier.fit(image_data,labels)
svm_classifier.score(image_data,labels)

0.6294240111034004